In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import warnings
import time
import ast
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from collections import Counter

# ball detection function
def detect_ball(image, hsv, lower_white, upper_white, lower_orange, upper_orange, 
                param1, param2, min_radius, max_radius, min_x, max_x, min_y, max_y, num_cells):
    """Detects white/orange balls and identifies (but ignores) large white eggs."""
    
    # Detect white ball (table tennis size)
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    blurred_white = cv2.GaussianBlur(mask_white, (9, 9), 2)
    circles_white = cv2.HoughCircles(blurred_white, cv2.HOUGH_GRADIENT, 1, 20, 
                                   param1=param1, param2=param2, 
                                   minRadius=min_radius, maxRadius=max_radius)

    # Detect orange ball
    mask_orange = cv2.inRange(hsv, lower_orange, upper_orange)
    blurred_orange = cv2.GaussianBlur(mask_orange, (9, 9), 2)
    circles_orange = cv2.HoughCircles(blurred_orange, cv2.HOUGH_GRADIENT, 1, 20, 
                                    param1=param1, param2=param2, 
                                    minRadius=min_radius, maxRadius=max_radius)

    # Detect large white egg (3x size of normal ball) - FOR VISUALIZATION ONLY
    egg_min_radius = max_radius * 2  # Minimum size for egg detection
    egg_max_radius = max_radius * 4  # Maximum size for egg detection
    circles_egg = cv2.HoughCircles(blurred_white, cv2.HOUGH_GRADIENT, 1, 40, 
                                 param1=param1, param2=max(10, param2-15),  # More tolerant circle detection
                                 minRadius=egg_min_radius, maxRadius=egg_max_radius)

    detected_balls = []
    egg_positions = []  # Will be used for obstacle avoidance
    
    # Process white balls (normal size)
    if circles_white is not None:
        circles_white = np.uint16(np.around(circles_white))
        for (x, y, r) in circles_white[0, :]:
            # Skip if this is actually part of an egg (nested circles)
            if circles_egg is not None:
                is_egg_part = any(np.sqrt((x-ex)**2 + (y-ey)**2) < er 
                              for (ex, ey, er) in circles_egg[0,:])
                if is_egg_part:
                    continue
            
            cv2.circle(image, (x, y), r, (255, 255, 255), 2)
            grid_col = int(((x - min_x) / (max_x - min_x)) * num_cells)
            grid_row = int(((y - min_y) / (max_y - min_y)) * num_cells)
            ball_position = (grid_col, grid_row)
            cv2.putText(image, f"w({grid_col}, {grid_row})", (x + 10, y - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            detected_balls.append(('white', ball_position))

    # Process orange balls
    if circles_orange is not None:
        circles_orange = np.uint16(np.around(circles_orange))
        for (x, y, r) in circles_orange[0, :]:
            cv2.circle(image, (x, y), r, (0, 165, 255), 2)
            grid_col = int(((x - min_x) / (max_x - min_x)) * num_cells)
            grid_row = int(((y - min_y) / (max_y - min_y)) * num_cells)
            ball_position = (grid_col, grid_row)
            cv2.putText(image, f"o({grid_col}, {grid_row})", (x + 10, y - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            detected_balls.append(('orange', ball_position))

    # Process and visualize large white egg (but don't add to detected_balls)
    if circles_egg is not None:
        circles_egg = np.uint16(np.around(circles_egg))
        for (x, y, r) in circles_egg[0, :]:
            # Draw egg with distinct visualization (red circle)
            cv2.circle(image, (x, y), r, (0, 0, 255), 3)
            grid_col = int(((x - min_x) / (max_x - min_x)) * num_cells)
            grid_row = int(((y - min_y) / (max_y - min_y)) * num_cells)
            egg_position = (grid_col, grid_row)
            cv2.putText(image, f"EGG({grid_col}, {grid_row})", (x + 20, y - 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            egg_positions.append(egg_position)

    return image, detected_balls, egg_positions  # Now returns egg positions for obstacle avoidance

In [5]:
import cv2
import numpy as np

def detect_eggs(frame, lower_white, upper_white, min_size=10000, max_size=30000):
    """
    Detects large white eggs (oval shapes) in a frame.
    
    Args:
        frame: Input BGR image
        lower_white: Lower HSV threshold for white color
        upper_white: Upper HSV threshold for white color
        min_size: Minimum pixel area for egg detection (default 10000)
        max_size: Maximum pixel area for egg detection (default 30000)
        
    Returns:
        result_frame: Frame with eggs marked
        egg_positions: List of ((center_x, center_y), (width, height), angle) for each egg
    """
    # Convert to HSV and create mask
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_white, upper_white)
    
    # Morphological operations to clean up mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    cleaned = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
    
    # Find contours
    contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Prepare output
    result_frame = frame.copy()
    egg_positions = []
    
    for contour in contours:
        area = cv2.contourArea(contour)
        
        # Size filtering for eggs
        if min_size < area < max_size:
            # Fit ellipse
            if len(contour) >= 5:  # Minimum points needed for ellipse fitting
                ellipse = cv2.fitEllipse(contour)
                
                # Aspect ratio check for oval shape (width/height between 1.2 and 2.5)
                (center, axes, angle) = ellipse
                width, height = axes
                aspect_ratio = max(width, height) / min(width, height)
                
                if 1.2 < aspect_ratio < 2.5:
                    # Draw ellipse
                    cv2.ellipse(result_frame, ellipse, (0, 0, 255), 3)
                    
                    # Draw center
                    center_x, center_y = map(int, center)
                    cv2.circle(result_frame, (center_x, center_y), 5, (0, 255, 0), -1)
                    
                    # Add label
                    cv2.putText(result_frame, f"EGG {int(width)}x{int(height)}", 
                               (center_x + 20, center_y), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, 
                               (0, 0, 255), 2)
                    
                    egg_positions.append(ellipse)
    
    return result_frame, egg_positions


def test_egg_detection():
    # HSV range for white eggs (adjust as needed)
    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 30, 255])
    
    cap = cv2.VideoCapture(0)
    print("Testing egg detection - Press 'q' to quit")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Detect eggs
        result, eggs = detect_eggs(frame, lower_white, upper_white)
        
        # Display
        cv2.imshow("Egg Detection", result)
        if eggs:
            print(f"Detected {len(eggs)} eggs")
            for i, ((x,y), (w,h), angle) in enumerate(eggs):
                print(f"Egg {i+1}: Center=({int(x)},{int(y)}), Size={int(w)}x{int(h)}, Angle={int(angle)}°")
        
        if cv2.waitKey(1) == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    test_egg_detection()

Testing egg detection - Press 'q' to quit
Detected 1 eggs
Egg 1: Center=(631,198), Size=134x217, Angle=2°
Detected 1 eggs
Egg 1: Center=(37,675), Size=160x305, Angle=169°
Detected 1 eggs
Egg 1: Center=(406,615), Size=135x185, Angle=127°
Detected 1 eggs
Egg 1: Center=(447,598), Size=184x270, Angle=68°
Detected 1 eggs
Egg 1: Center=(431,614), Size=168x254, Angle=97°
Detected 1 eggs
Egg 1: Center=(1076,1039), Size=154x320, Angle=84°
Detected 1 eggs
Egg 1: Center=(1076,1040), Size=156x320, Angle=84°
Detected 1 eggs
Egg 1: Center=(1076,1040), Size=157x322, Angle=84°
Detected 1 eggs
Egg 1: Center=(1076,1040), Size=155x322, Angle=84°
Detected 1 eggs
Egg 1: Center=(1075,1040), Size=155x325, Angle=83°
Detected 1 eggs
Egg 1: Center=(412,629), Size=166x267, Angle=102°
Detected 1 eggs
Egg 1: Center=(1077,1039), Size=154x322, Angle=84°
Detected 1 eggs
Egg 1: Center=(1076,1041), Size=158x319, Angle=84°
Detected 1 eggs
Egg 1: Center=(1075,1041), Size=157x318, Angle=84°
Detected 1 eggs
Egg 1: Center=(

In [1]:
import cv2
import numpy as np

def test_cross_detection():
    # Initialize camera
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera")
        return
    
    # Create trackbar window
    cv2.namedWindow("Cross Detection Test")
    cv2.resizeWindow("Cross Detection Test", 800, 600)
    
    # Default HSV values for red detection
    cv2.createTrackbar("Hue Min", "Cross Detection Test", 0, 179, lambda x: None)
    cv2.createTrackbar("Hue Max", "Cross Detection Test", 10, 179, lambda x: None)
    cv2.createTrackbar("Sat Min", "Cross Detection Test", 100, 255, lambda x: None)
    cv2.createTrackbar("Sat Max", "Cross Detection Test", 255, 255, lambda x: None)
    cv2.createTrackbar("Val Min", "Cross Detection Test", 100, 255, lambda x: None)
    cv2.createTrackbar("Val Max", "Cross Detection Test", 255, 255, lambda x: None)
    cv2.createTrackbar("Min Area", "Cross Detection Test", 300, 1000, lambda x: None)
    cv2.createTrackbar("Max Area", "Cross Detection Test", 1000, 5000, lambda x: None)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame")
            break
        
        # Get current trackbar positions
        h_min = cv2.getTrackbarPos("Hue Min", "Cross Detection Test")
        h_max = cv2.getTrackbarPos("Hue Max", "Cross Detection Test")
        s_min = cv2.getTrackbarPos("Sat Min", "Cross Detection Test")
        s_max = cv2.getTrackbarPos("Sat Max", "Cross Detection Test")
        v_min = cv2.getTrackbarPos("Val Min", "Cross Detection Test")
        v_max = cv2.getTrackbarPos("Val Max", "Cross Detection Test")
        min_area = cv2.getTrackbarPos("Min Area", "Cross Detection Test")
        max_area = cv2.getTrackbarPos("Max Area", "Cross Detection Test")
        
        # Convert to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Create masks for red color (two ranges)
        lower_red1 = np.array([h_min, s_min, v_min])
        upper_red1 = np.array([h_max, s_max, v_max])
        lower_red2 = np.array([160, s_min, v_min])
        upper_red2 = np.array([179, s_max, v_max])
        
        mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
        mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        red_mask = cv2.bitwise_or(mask1, mask2)
        
        # Morphological operations
        kernel = np.ones((5,5), np.uint8)
        cleaned = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel)
        cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel)
        
        # Find contours
        contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Process contours
        cross_found = False
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if min_area < area < max_area:
                # Check for cross-like shape
                peri = cv2.arcLength(cnt, True)
                approx = cv2.approxPolyDP(cnt, 0.04 * peri, True)
                
                if len(approx) >= 8:  # Cross has many vertices
                    M = cv2.moments(cnt)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        
                        # Draw contour and center
                        cv2.drawContours(frame, [cnt], -1, (0,255,0), 2)
                        cv2.circle(frame, (cx, cy), 5, (255,0,0), -1)
                        cv2.putText(frame, "CROSS", (cx+10, cy), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                        cross_found = True
        
        # Show mask if no cross found
        if not cross_found:
            cv2.putText(frame, "No cross detected", (20, 40), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        
        # Display results
        cv2.imshow("Cross Detection Test", frame)
        cv2.imshow("Mask", cleaned)
        
        # Key controls
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            print(f"Current HSV values: Lower1={lower_red1}, Upper1={upper_red1}")
            print(f"                Lower2={lower_red2}, Upper2={upper_red2}")
            print(f"Area range: {min_area}-{max_area}")
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    test_cross_detection()

2025-06-20 10:52:22.247 python[84754:3254100] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
